# Les bases de données 


## Introduction 

Comme on a pu le voir dans les diapos de cours, une base de données est **un conteneur dans lequel il est possible de stocker des données de façon structurée.** Cette structure permet au programme informatique connectée à celle-ci de faire des recherches complexes.


## Et python dans tout ca ! 


### SQLite
Notre exemple de cours concernera le module python SQLite et plus particulièrement SQLite3.
SQLite a été conçu pour être intégré dans le programme même. Pour des projets plus ambitieux type projets web ou infrastructure le choix de MySQL & SQL serait plus judicieux. 


In [12]:
import sqlite3

#### Créer une base de données avec SQLite :

In [13]:
conn = sqlite3.connect('./db/ma_base_exemple.db')

Lorsque vous executerez votre programme vous remarquerez que si la base n'existe pas encore, un fichier sera crée dans le dossier de votre programme. Et si celui-ci existe déjà il sera réutilisé. Vous pouvez bien évidemment choisir l'emplacement de votre base de données en renseignant un path, exemple: "/data/ma_base_exemple.db" . *Il vous faudra cependant vérifier que le dossier existe avant de l'utiliser.*
Il est également possible de travailler avec une base de données de manière temporaire:

In [10]:
#Lorsque le travail que vous attendiez est terminé, pensez à fermer la connexion vers la base
#conn.close()

#### Un exemple de création de table : 

In [17]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS users(
     id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
     name TEXT,
     age INTERGER
)
""")
conn.commit()


#### Supprimer la table 

In [15]:
cursor = conn.cursor()
cursor.execute("""
DROP TABLE users
""")
conn.commit()

#### Insérer des data 

Il existe plusieurs manière d'insérer des données, la plus simple étant celle-ci :

In [18]:
cursor.execute("""
INSERT INTO users(name, age) VALUES(?, ?)""", ("olivier", 30))

#### Vous pouvez passer par un dictionnaire :

In [19]:
data = {"name" : "olivier", "age" : 30}
cursor.execute("""
INSERT INTO users(name, age) VALUES(:name, :age)""", data)

#### Vous pouvez récupérer l'id de la ligne que vous venez d'insérer de cette manière :

In [20]:
id = cursor.lastrowid
print('dernier id: %d' % id)

dernier id: 2


#### Il est également possible de faire plusieurs insert en une seule fois avec la fonction executemany :


In [21]:
users = []
users.append(("olivier", 30))
users.append(("jean-louis", 90))
cursor.executemany("""
INSERT INTO users(name, age) VALUES(?, ?)""", users)

#### Récupérer des données

Vous pouvez récupérer la première ligne correspondant à votre recherche à l'aide de **la fonction fetchone.**

In [23]:
cursor.execute("""SELECT name, age FROM users""")
user1 = cursor.fetchone()
#print le TUPLE
print(user1)

('olivier', 30)


#### Récupérer plusieur data 
Vous pouvez récupérer plusieurs données de la même recherche en utilisant **la fonction fetchall().**

In [24]:
cursor.execute("""SELECT id, name, age FROM users""")
rows = cursor.fetchall()
for row in rows:
    print('{0} : {1} - {2}'.format(row[0], row[1], row[2]))

1 : olivier - 30
2 : olivier - 30
3 : olivier - 30
4 : jean-louis - 90


In [25]:
#L'objet curseur fonctionne comme un itérateur, invoquant la méthode fetchall() automatiquement 
cursor.execute("""SELECT id, name, age FROM users""")
for row in cursor:
    print('{0} : {1}, {2}'.format(row[0], row[1], row[2]))


1 : olivier, 30
2 : olivier, 30
3 : olivier, 30
4 : jean-louis, 90


In [27]:
#Pour la recherche spécifique,on utilise la même logique vu précédemment :
id = 2
cursor.execute("""SELECT id, name FROM users WHERE id=?""", (id,))
response = cursor.fetchone()

#### Gestion des erreurs 


In [28]:
try:
    conn = sqlite3.connect('db/users.db')
    cursor = conn.cursor()
    cursor.execute("""
CREATE TABLE users(
    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT,
    age INTERGER
)
""")
    conn.commit()
except sqlite3.OperationalError:
    print('Erreur la table existe déjà')
except Exception as e:
    print("Erreur")
    conn.rollback()
    # raise e
finally:
    conn.close()

Erreur la table existe déjà


Les erreurs que vous pouvez intercepter :
```
Error
DatabaseError
DataError
IntegrityError
InternalError
NotSupportedError
OperationalError
ProgrammingError
InterfaceError
Warning
```